# Image Classification with TensorFlow Lite Model Maker

## Prerequisites

To run this example, we first need to install several required packages, including Model Maker package that in GitHub [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker

Import the required packages.

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

## Simple End-to-End Example

Step 1.   Load input data specific to an on-device ML app. Split it into training data and testing data.

In [ ]:
os.mkdir("database")
!unzip database.zip -d database/
image_path="database"

In [ ]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

Step 2. Customize the TensorFlow model.

In [ ]:
model = image_classifier.create(train_data, epochs=50)

Step 3. Evaluate the model.

In [ ]:
loss, accuracy = model.evaluate(test_data)

Step 4.  Export to TensorFlow Lite model.



In [ ]:
model.export(export_dir='.', tflite_filename='model.tflite')

Step 5. Export to labels.txt

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.LABEL)

#**---------------------------------------**

# Testing TF Lite Model

In [ ]:
!pip install opencv-python-headless==4.5.2.52

In [60]:
import tensorflow.lite as tflite
import cv2
import numpy as np

In [61]:
# Load TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path='model.tflite')

with open("labels.txt", 'r') as f:
    labels = [line.strip() for line in f.readlines()]

#allocate the tensors
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [62]:
def reader(image_path):
    #image name here
    img = cv2.imread(image_path)
    img = cv2.resize(img,(224, 224))
    #Preprocess the image to required size and castinput_shape = input_details[0]['shape']
    input_tensor= np.array(np.expand_dims(img,0))
    input_tensor=input_tensor.astype('uint8') #uint8 for colab trained, float32 for normal

    input_index = interpreter.get_input_details()[0]["index"]
    interpreter.set_tensor(input_index, input_tensor)
    interpreter.invoke()
    output_details = interpreter.get_output_details()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    pred = np.squeeze(output_data)

    highest_pred_loc = np.argmax(pred)
    object_name = labels[highest_pred_loc-1]
    print(f"It's a {object_name}.")

In [ ]:
reader("test.jpg")